In [ ]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import KFold
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
import csv
import pandas as pd
import os
import nltk
from nltk.corpus import stopwords
import string
import seaborn as sns
from sklearn.pipeline import make_pipeline
import nltk
nltk.download('stopwords')
nltk.download('punkt')
%matplotlib inline


df = pd.read_csv("fake_or_real_news.csv", sep = ",")

In [ ]:
a = pd.read_csv("fake_or_real_news.csv", sep = ",")
b1 = pd.read_csv("Fake.csv", sep = ",")
b2 = pd.read_csv("True.csv", sep = ",")
c = pd.read_csv("data.csv", sep = ",")

In [ ]:
c['title_text_source'] = c['Headline'] + ' ' + c['Body']

list_c = []
for x in c['Label']:
    if x==1:
        list_c.append("REAL")
    elif x==0:
        list_c.append("FAKE")

c["label"] = list_c
del c["Label"]

c

In [ ]:
a['title_text_source'] = a['title'] + ' ' + a['text']
c = c.fillna("")

In [ ]:
labels = []
for x in range(len(b1)):
    labels.append("FAKE")
    
b1["label"] = labels

labels2 = []
for x in range(len(b2)):
    labels2.append("REAL")
    
b2["label"] = labels2

print(len(b1) + len(b2))

b = pd.concat([b1, b2], ignore_index=True)

b['title_text_source'] = b['title'] + ' ' + b['text']

b.head()

In [ ]:
a = a[a['label']!='']
print(a['label'].unique())

b = b[b['label']!='']
print(b['label'].unique())

c = c[c['label']!='']
print(c['label'].unique())

In [ ]:
a.loc[a['label'] == 'fake', 'label'] = 'FAKE'
a.loc[a['label'] == 'Fake', 'label'] = 'FAKE'

no_of_fakes = a.loc[a['label'] == 'FAKE'].count()[0]
no_of_reals = a.loc[a['label'] == 'REAL'].count()[0]
print(no_of_fakes)
print(no_of_reals)
print("\n")


b.loc[b['label'] == 'fake', 'label'] = 'FAKE'
b.loc[b['label'] == 'Fake', 'label'] = 'FAKE'

no_of_fakes_b = b.loc[b['label'] == 'FAKE'].count()[0]
no_of_reals_b = b.loc[b['label'] == 'REAL'].count()[0]
print(no_of_fakes_b)
print(no_of_reals_b)
print("\n")

c.loc[c['label'] == 'fake', 'label'] = 'FAKE'
c.loc[c['label'] == 'Fake', 'label'] = 'FAKE'

no_of_fakes_c = c.loc[c['label'] == 'FAKE'].count()[0]
no_of_reals_c = c.loc[c['label'] == 'REAL'].count()[0]
print(no_of_fakes_c)
print(no_of_reals_c)

In [ ]:
stop_words = set(stopwords.words('english'))
def clean(text):    
    # Lowering letters
    text = text.lower()
    
    # Removing html tags
    text = re.sub(r'<[^>]*>', '', text)
    
    # Removing twitter usernames
    text = re.sub(r'@[A-Za-z0-9]+','',text)
    
    # Removing urls
    text = re.sub('https?://[A-Za-z0-9]','',text)
    
    # Removing numbers
    text = re.sub('[^a-zA-Z]',' ',text)
    
    word_tokens = word_tokenize(text)
    
    filtered_sentence = []
    for word_token in word_tokens:
        if word_token not in stop_words:
            filtered_sentence.append(word_token)
    
    # Joining words
    text = (' '.join(filtered_sentence))
    return text

In [ ]:
a['title_text_source'] = a['title_text_source'].apply(clean)
b['title_text_source'] = b['title_text_source'].apply(clean)
c['title_text_source'] = c['title_text_source'].apply(clean)

In [ ]:
vectorizer = CountVectorizer()
X_a = vectorizer.fit_transform(a['title_text_source'].values)
X_a = X_a.toarray()

X_b = vectorizer.transform(b['title_text_source'].values)
X_c = vectorizer.transform(c['title_text_source'].values)

y_a = a['label'].values
y_b = b['label'].values
y_c = c['label'].values

In [ ]:
x_list = [X_a, X_b, X_c]
y_list = [y_a, y_b, y_c]

In [ ]:
df['title_text_source'] = df['title'] + ' ' + df['text']
df.head()

In [ ]:
df = df[df['label']!='']
print(df['label'].unique())

In [ ]:
df.loc[df['label'] == 'fake', 'label'] = 'FAKE'
df.loc[df['label'] == 'Fake', 'label'] = 'FAKE'

no_of_fakes = df.loc[df['label'] == 'FAKE'].count()[0]
no_of_reals = df.loc[df['label'] == 'REAL'].count()[0]
print(no_of_fakes)
print(no_of_reals)

In [ ]:
df['title_text_source'] = df['title_text_source'].apply(clean)

In [ ]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['title_text_source'].values)
X = X.toarray()

In [ ]:
import seaborn as sns

## Random Forest Cross-Fold Validation

In [ ]:
y = df['label'].values
kf = KFold(n_splits=5)

cv_results = {
    "sensitivity" : [],
    "specificity" : [],
    "accuracy" : [],
    "npv" : [],
    "precision" : [],
    "cm" : [],
    "f1" : []
}

for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    
    clf = RandomForestClassifier(n_estimators = 200)
    clf.fit(X_train, y_train)

    predictions = clf.predict(X_test)

    cm = confusion_matrix(y_test, predictions)
    
    sensitivity = cm[1][1] / (cm[1][1] + cm[1][0])
    specificity = cm[0][0] / (cm[0][0] + cm[0][1])
    accuracy = (cm[1][1] + cm[0][0]) / (cm[0][0] + cm[0][1] + cm[1][0] + cm[1][1])
    negative = cm[0][0] / (cm[0][0] + cm[1][0])
    precision = cm[1][1] / (cm[1][1] + cm[0][1])
    f1 = (2 * precision * sensitivity) / (precision + sensitivity)
    
    cv_results["sensitivity"].append(sensitivity)
    cv_results["specificity"].append(specificity)
    cv_results["accuracy"].append(accuracy)
    cv_results["npv"].append(negative)
    cv_results["precision"].append(precision)
    cv_results["cm"].append(cm)
    cv_results["f1"].append(f1)

In [ ]:
cm_mean = sum(cv_results["cm"]) / 5

plt.figure(figsize=(6,6))
sns.heatmap(cm_mean, annot=True, fmt = ".1f", xticklabels = ['FAKE', 'REAL'], yticklabels = ['FAKE', 'REAL'], cmap = plt.cm.Blues, cbar = False).set(title='Cross Validation Accuray - Random Forest')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:
print(f"""sensitivity: {np.mean(cv_results['sensitivity']):.3f} ± {np.std(cv_results['sensitivity']):.3f}""")
print(f"""specificity: {np.mean(cv_results['specificity']):.2f} ± {np.std(cv_results['specificity']):.2f}""")
print(f"""accuracy: {np.mean(cv_results['accuracy']):.3f} ± {np.std(cv_results['accuracy']):.3f}""")
print(f"""npv: {np.mean(cv_results['npv']):.3f} ± {np.std(cv_results['npv']):.3f}""")
print(f"""precision: {np.mean(cv_results['precision']):.2f} ± {np.std(cv_results['precision']):.2f}""")
print(f"""f1: {np.mean(cv_results['f1']):.3f} ± {np.std(cv_results['f1']):.3f}""")

## Random Forest External Validation

In [ ]:
clf = RandomForestClassifier(n_estimators = 10)

cv_results = {
    "sensitivity" : [],
    "specificity" : [],
    "accuracy" : [],
    "npv" : [],
    "precision" : [],
    "cm" : [],
    "f1" : []
}

for i in range(len(x_list)):
    clf.fit(x_list[i], y_list[i])
    for j in range (len(x_list)):
        predictions = clf.predict(x_list[j])
        cm = confusion_matrix(y_list[j], predictions)
        accuracy = (cm[1][1] + cm[0][0]) / (cm[0][0] + cm[0][1] + cm[1][0] + cm[1][1])
        
        first = ""
        second = ""
        
        if i == 0:
            first = "A"
        elif i == 1:
            first = "B"
        elif i == 2:
            first = "C"
            
        if j == 0:
            second = "A"
        elif j == 1:
            second = "B"
        elif j == 2:
            second = "C"
        
        print("Fit on " + first + ", Tested on " + second + " - accuracy: " + str(accuracy))

In [ ]:
list = [[0.9951065509076559, 0.6265535213149807, 0.6384730538922155], [0.5696921862667719, 0.9995545458595038, 0.7118263473053892], [0.5829518547750592, 0.6372666933939151, 0.9987524950099801]]
map = sns.heatmap(list, xticklabels=["a", "b", "c"], yticklabels=["a", "b", "c"], annot=True, vmin=0, vmax=1, cmap="Blues")
map = map.set(xlabel='Testing Set', ylabel='Training Set', title='External Validation Accuray - Random Forest')
map

## Naive Bayes Cross-Fold Validation

In [ ]:
y = df['label'].values
kf = KFold(n_splits=5)

cv_results = {
    "sensitivity" : [],
    "specificity" : [],
    "accuracy" : [],
    "npv" : [],
    "precision" : [],
    "cm" : [],
    "f1" : []
}

for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    
    clf = MultinomialNB()
    clf.fit(X_train, y_train)

    predictions = clf.predict(X_test)

    cm = confusion_matrix(y_test, predictions)
    
    sensitivity = cm[1][1] / (cm[1][1] + cm[1][0])
    specificity = cm[0][0] / (cm[0][0] + cm[0][1])
    accuracy = (cm[1][1] + cm[0][0]) / (cm[0][0] + cm[0][1] + cm[1][0] + cm[1][1])
    negative = cm[0][0] / (cm[0][0] + cm[1][0])
    precision = cm[1][1] / (cm[1][1] + cm[0][1])
    f1 = (2 * precision * sensitivity) / (precision + sensitivity)
    
    cv_results["sensitivity"].append(sensitivity)
    cv_results["specificity"].append(specificity)
    cv_results["accuracy"].append(accuracy)
    cv_results["npv"].append(negative)
    cv_results["precision"].append(precision)
    cv_results["cm"].append(cm)
    cv_results["f1"].append(f1)

In [ ]:
cm_mean = sum(cv_results["cm"]) / 5

plt.figure(figsize=(6,6))
sns.heatmap(cm_mean, annot=True, fmt = ".1f", xticklabels = ['FAKE', 'REAL'], yticklabels = ['FAKE', 'REAL'], cmap = plt.cm.Blues, cbar = False).set(title='Cross Validation Accuray - Naive Bayes')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:
print(f"""sensitivity: {np.mean(cv_results['sensitivity']):.3f} ± {np.std(cv_results['sensitivity']):.3f}""")
print(f"""specificity: {np.mean(cv_results['specificity']):.2f} ± {np.std(cv_results['specificity']):.2f}""")
print(f"""accuracy: {np.mean(cv_results['accuracy']):.3f} ± {np.std(cv_results['accuracy']):.3f}""")
print(f"""npv: {np.mean(cv_results['npv']):.3f} ± {np.std(cv_results['npv']):.3f}""")
print(f"""precision: {np.mean(cv_results['precision']):.2f} ± {np.std(cv_results['precision']):.2f}""")
print(f"""f1: {np.mean(cv_results['f1']):.3f} ± {np.std(cv_results['f1']):.3f}""")

## Naive Bayes External Validation

In [ ]:
clf = MultinomialNB()

for i in range(len(x_list)):
    clf.fit(x_list[i], y_list[i])
    for j in range (len(x_list)):
        predictions = clf.predict(x_list[j])
        cm = confusion_matrix(y_list[j], predictions)
        accuracy = (cm[1][1] + cm[0][0]) / (cm[0][0] + cm[0][1] + cm[1][0] + cm[1][1])
        
        first = ""
        second = ""
        
        if i == 0:
            first = "A"
        elif i == 1:
            first = "B"
        elif i == 2:
            first = "C"
            
        if j == 0:
            second = "A"
        elif j == 1:
            second = "B"
        elif j == 2:
            second = "C"
        
        print("Fit on " + first + ", Tested on " + second + " - accuracy: " + str(accuracy))

In [ ]:
list = [[0.9363851617995265, 0.575148113501715, 0.5805888223552894], [0.5753749013417522, 0.9572141298053365, 0.7200598802395209], [0.6393054459352802, 0.6000044545414049, 0.9807884231536926]]
map = sns.heatmap(list, xticklabels=["a", "b", "c"], yticklabels=["a", "b", "c"], annot=True, vmin=0, vmax=1, cmap="Blues")
map = map.set(xlabel='Testing Set', ylabel='Training Set', title='External Validation Accuray - Random Forest')
map

## SVM Cross-Fold Validation

In [ ]:
y = df['label'].values
kf = KFold(n_splits=5)

cv_results = {
    "sensitivity" : [],
    "specificity" : [],
    "accuracy" : [],
    "npv" : [],
    "precision" : [],
    "cm" : [],
    "f1" : []
}

for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    
    clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
    clf.fit(X_train, y_train)

    predictions = clf.predict(X_test)

    cm = confusion_matrix(y_test, predictions)
    
    sensitivity = cm[1][1] / (cm[1][1] + cm[1][0])
    specificity = cm[0][0] / (cm[0][0] + cm[0][1])
    accuracy = (cm[1][1] + cm[0][0]) / (cm[0][0] + cm[0][1] + cm[1][0] + cm[1][1])
    negative = cm[0][0] / (cm[0][0] + cm[1][0])
    precision = cm[1][1] / (cm[1][1] + cm[0][1])
    f1 = (2 * precision * sensitivity) / (precision + sensitivity)
    
    cv_results["sensitivity"].append(sensitivity)
    cv_results["specificity"].append(specificity)
    cv_results["accuracy"].append(accuracy)
    cv_results["npv"].append(negative)
    cv_results["precision"].append(precision)
    cv_results["cm"].append(cm)
    cv_results["f1"].append(f1)

In [ ]:
cm_mean = sum(cv_results["cm"]) / 5

plt.figure(figsize=(6,6))
sns.heatmap(cm_mean, annot=True, fmt = ".1f", xticklabels = ['FAKE', 'REAL'], yticklabels = ['FAKE', 'REAL'], cmap = plt.cm.Blues, cbar = False).set(title='Cross Validation Accuray - SVM')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:
print(f"""sensitivity: {np.mean(cv_results['sensitivity']):.3f} ± {np.std(cv_results['sensitivity']):.3f}""")
print(f"""specificity: {np.mean(cv_results['specificity']):.2f} ± {np.std(cv_results['specificity']):.2f}""")
print(f"""accuracy: {np.mean(cv_results['accuracy']):.3f} ± {np.std(cv_results['accuracy']):.3f}""")
print(f"""npv: {np.mean(cv_results['npv']):.3f} ± {np.std(cv_results['npv']):.3f}""")
print(f"""precision: {np.mean(cv_results['precision']):.2f} ± {np.std(cv_results['precision']):.2f}""")
print(f"""f1: {np.mean(cv_results['f1']):.3f} ± {np.std(cv_results['f1']):.3f}""")

## SVM External Validation

In [ ]:
vectorizer = CountVectorizer()

X_a = vectorizer.fit_transform(a['title_text_source'].values)
X_a = X_a.toarray()

X_b = vectorizer.transform(b['title_text_source'].values)
X_b = X_b.toarray()

X_c = vectorizer.transform(c['title_text_source'].values)
X_c = X_c.toarray()

y_a = a['label'].values
y_b = b['label'].values
y_c = c['label'].values

In [ ]:
x_list = [X_a, X_c]
y_list = [y_a, y_c]

In [ ]:
clf = make_pipeline(StandardScaler(with_mean=False), SVC(gamma='auto'))

In [ ]:
for i in range(len(x_list)):
    clf.fit(x_list[i], y_list[i])
    for j in range (len(x_list)):
        predictions = clf.predict(x_list[j])
        cm = confusion_matrix(y_list[j], predictions)
        accuracy = (cm[1][1] + cm[0][0]) / (cm[0][0] + cm[0][1] + cm[1][0] + cm[1][1])
        
        first = ""
        second = ""
        
        if i == 0:
            first = "A"
        elif i == 1:
            first = "C"
            
        if j == 0:
            second = "A"
        elif j == 1:
            second = "C"
        
        print("Fit on " + first + ", Tested on " + second + " - accuracy: " + str(accuracy))

In [ ]:
Fit on A, Tested on A - accuracy: 0.9403314917127071
Fit on A, Tested on C - accuracy: 0.5765968063872255
Fit on C, Tested on A - accuracy: 0.5957379636937647
Fit on C, Tested on C - accuracy: 0.9239021956087824

In [ ]:
list = [[0.9403314917127071, 0.5765968063872255], [0.5957379636937647, 0.9239021956087824]]
map = sns.heatmap(list, xticklabels=["a", "c"], yticklabels=["a", "c"], annot=True, vmin=0, vmax=1, cmap="Blues")
map = map.set(xlabel='Testing Set', ylabel='Training Set', title='External Validation Accuray - SVM')
map